# Bronze DDL
## Working approach
For sake of semplicity, I created 4 bronze tables in bronze schema by uploading the CSV file directly. You can scroll to the end of the notebook for some additional input on how I would have managed the ingestion from a Postgre SQL instance.

The script below requires an existing catalog (ideally created with Terraform script but more likely manually if the platform is not already in a very mature state) and contains the creation statement of the schema and the 4 tables. 
It can be run during deployment through deployment pipeline execution as the environment is a parameter passed.
I did not change the data types to the proper one in this phase (it will be performed during bronze-->silver transformation, more reason for that at the end), just uploaded the data as it is in the csv.


In [0]:
dbutils.widgets.text("env", "dev")

In [0]:
environment = dbutils.widgets.get('env')
catalog_name = "use_case_" + environment
schema_name = "bronze"

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog_name}.{schema_name}")
# For automatic execution, we can add commands for granting/managing privileges on schema AND tables

In [0]:
spark.sql(f"""CREATE TABLE {catalog_name}.{schema_name}.teams (
  team_id STRING,
  team_activity STRING,
  country_code STRING,
  created_at STRING)
USING delta
TBLPROPERTIES (
  'delta.minReaderVersion' = '1',
  'delta.minWriterVersion' = '2')
  """)

In [0]:
spark.sql(f"""CREATE TABLE {catalog_name}.{schema_name}.memberships (
  membership_id STRING,
  group_id STRING,
  role_title STRING,
  joined_at STRING)
USING delta
TBLPROPERTIES (
  'delta.minReaderVersion' = '1',
  'delta.minWriterVersion' = '2')
  """)

In [0]:
spark.sql(f"""CREATE TABLE {catalog_name}.{schema_name}.events (
  event_id STRING,
  team_id STRING,
  event_start STRING,
  event_end STRING,
  latitude DOUBLE,
  longitude DOUBLE,
  created_at STRING)
USING delta
TBLPROPERTIES (
  'delta.minReaderVersion' = '1',
  'delta.minWriterVersion' = '2')
  """)

In [0]:
spark.sql(f"""CREATE TABLE {catalog_name}.{schema_name}.event_rsvps (
  event_rsvp_id STRING,
  event_id STRING,
  membership_id STRING,
  rsvp_status BIGINT,
  responded_at STRING)
USING delta
TBLPROPERTIES (
  'delta.minReaderVersion' = '1',
  'delta.minWriterVersion' = '2')
  """)

## Lakehouse federation for PostGreSQL sources

In a real case scenario (databricks enviroment with UC enabled, cloud instance for the database), to approach this use case I would use the "relatively new" Lakehouse Federation framework.

In this approach, you create a connection object directly in UC:

`CREATE CONNECTION <connection-name> TYPE postgresql
OPTIONS (
  host '<hostname>',
  port '<port>',
  user secret ('<secret-scope>','<secret-key-user>'),
  password secret ('<secret-scope>','<secret-key-password>')
)`

and a foreign catalog using that connection to "federate" the postgre sql instance under Databricks UC governance capabilities.

`CREATE FOREIGN CATALOG [IF NOT EXISTS] <catalog-name> USING CONNECTION <connection-name>
OPTIONS (database '<database-name>')`


with this in place, it would be possible directly query the data in posgre by creating a materialized view or using the table under the foreign catalog as sources for a DLT pipeline.